In [ ]:
#generate single digit fraction
import random
import os
from PIL import Image, ImageDraw
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor, Resize, ToPILImage

mnist_dataset = MNIST(root=".", train=True, download=True, transform=ToTensor())

# Step 2: Generate fraction pairs
num_pairs = 3000 

fraction_pairs = []
for _ in range(num_pairs):
    numerator_digit = random.randint(1, 9)  
    denominator_digit_index = random.randint(0, len(mnist_dataset) - 1) 
    denominator_digit_image = mnist_dataset[denominator_digit_index][0]
    
    numerator_images = [mnist_dataset[numerator_digit][0]] 
    denominator_images = [denominator_digit_image]
    
    fraction_pairs.append((numerator_images, denominator_images))

# Step 3: Create fraction images
image_width = 56
image_height = 56

fraction_images = []
for numerator_images, denominator_images in fraction_pairs:
    numerator_images_resized = []
    denominator_images_resized = []
    
    # Resize numerator 
    for digit_image in numerator_images:
        digit_image_resized = Resize((image_width // 2, image_height // 2))(digit_image)
        numerator_images_resized.append(digit_image_resized)
    
    # Resize denominator 
    for digit_image in denominator_images:
        digit_image_resized = Resize((image_width // 2, image_height // 2))(digit_image)
        denominator_images_resized.append(digit_image_resized)
    
    # Create the fraction image
    image = Image.new("RGB", (image_width, image_height), (255, 255, 255))
    
    # height
    digit_height = numerator_images_resized[0].shape[1]
    
    # y-coordinate 
    digits_y = (image_height - digit_height * 2) // 2
    
    # Paste numerator digits
    x_offset = (image_width - numerator_images_resized[0].shape[2]) // 2
    for digit_image in numerator_images_resized:
        digit_pil_image = ToPILImage()(digit_image)
        width, height = digit_pil_image.size
        image.paste(digit_pil_image, (x_offset, digits_y, x_offset + width, digits_y + height))
        x_offset += width
    
  
    x_offset = (image_width - denominator_images_resized[0].shape[2]) // 2
    for digit_image in denominator_images_resized:
        digit_pil_image = ToPILImage()(digit_image)
        width, height = digit_pil_image.size
        image.paste(digit_pil_image, (x_offset, digits_y + digit_height, x_offset + width, digits_y + digit_height + height))
        x_offset += width
    
    # Draw the fraction bar
    fraction_bar_y = image_height // 2
    fraction_bar_height = 2
    fraction_bar_x_start = 0
    fraction_bar_x_end = image_width
    draw = ImageDraw.Draw(image)
    draw.rectangle([(fraction_bar_x_start, fraction_bar_y - fraction_bar_height // 2), (fraction_bar_x_end, fraction_bar_y + fraction_bar_height // 2)], fill=(255, 255, 255))
    
    fraction_images.append(image)

output_directory = "fraction_single/"

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

for i, image in enumerate(fraction_images):
    image_rescaled = Resize((56, 56))(image)
    image_rescaled.save(output_directory + f"fraction_{i}.png")


In [ ]:
#generate two digits fraction
import random
import os
from PIL import Image, ImageDraw
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor, ToPILImage, Resize

mnist_dataset = MNIST(root=".", train=True, download=True, transform=ToTensor())

num_pairs = 3000
fraction_pairs = []
for _ in range(num_pairs):
    numerator_digits = random.sample(range(10), 2)
    denominator_digits = random.sample(range(10), 2)

    numerator_images = []
    denominator_images = []

    for digit in numerator_digits:
        digit_index = random.randint(0, len(mnist_dataset) - 1)
        digit_image = mnist_dataset[digit_index][0]
        numerator_images.append(digit_image)

    for digit in denominator_digits:
        digit_index = random.randint(0, len(mnist_dataset) - 1)
        digit_image = mnist_dataset[digit_index][0]
        denominator_images.append(digit_image)

    fraction_pairs.append((numerator_images, denominator_images))

image_width = 56
image_height = 56

fraction_images = []
for numerator_images, denominator_images in fraction_pairs:

    resized_numerator_images = [Resize((28, 28))(ToPILImage()(digit_image)) for digit_image in numerator_images]
    resized_denominator_images = [Resize((28, 28))(ToPILImage()(digit_image)) for digit_image in denominator_images]

    image = Image.new("L", (image_width, image_height), (255))
    draw = ImageDraw.Draw(image)

    x_offset = 0
    for digit_image in resized_numerator_images:
        width, height = digit_image.size
        image.paste(digit_image, (x_offset, 0, x_offset + width, height))
        x_offset += width

    fraction_bar_y = image_height // 2 - 1 
    fraction_bar_x_start = 0
    fraction_bar_x_end = image_width
    fraction_bar_height = 3  
    draw.rectangle([(fraction_bar_x_start, fraction_bar_y), (fraction_bar_x_end, fraction_bar_y + fraction_bar_height)], fill=(255))

    x_offset = 0
    for digit_image in resized_denominator_images:
        width, height = digit_image.size
        image.paste(digit_image, (x_offset, image_height - height, x_offset + width, image_height))
        x_offset += width

    fraction_images.append(image)

output_directory = "two_digits_fraction/"

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

for i, image in enumerate(fraction_images):
    image.save(output_directory + f"fraction_{i}.png")


In [ ]:
from PIL import Image, ImageOps
import os

def invert_image_colors(image_path):
    # Load the image
    image = Image.open(image_path)
    
    # Convert the colors
    inverted_image = ImageOps.invert(image)
    
    return inverted_image

def process_directory(input_directory, output_directory):
    for root, dirs, files in os.walk(input_directory):
        for file in files:
            if file.endswith(".jpg") or file.endswith(".png"):
                input_path = os.path.join(root, file)
                
                relative_path = os.path.relpath(root, input_directory)
                output_path_root = os.path.join(output_directory, relative_path)
                if not os.path.exists(output_path_root):
                    os.makedirs(output_path_root)
                output_path = os.path.join(output_path_root, file)
                
                inverted_image = invert_image_colors(input_path)
                inverted_image.save(output_path)

input_directory = 'two_digits_fraction'
output_directory = 'converted_two_digits_fraction'
process_directory(input_directory, output_directory)